## Load Data Set

In [2]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical


print(tf.__version__)

root_path = './'
dataset_path = os.path.join(root_path, 'tf_dataset')

models_path = os.path.join(root_path, 'saved_models')
if not os.path.exists(models_path):
  os.mkdir(models_path)

Using TensorFlow backend.


1.13.1


In [3]:
BATCH_SIZE = 2


image_feature_description = {
    'enc': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    'img': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True)
}


def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)


raw_train_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path,'train.tfrecords'))
parsed_train_dataset = raw_train_dataset.map(_parse_image_function)

raw_val_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'val.tfrecords'))
parsed_val_dataset = raw_val_dataset.map(_parse_image_function)

raw_test_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'test.tfrecords'))
parsed_test_dataset = raw_test_dataset.map(_parse_image_function)


parsed_train_dataset = parsed_train_dataset.repeat()
parsed_train_dataset = parsed_train_dataset.shuffle(1000)
parsed_train_dataset = parsed_train_dataset.batch(BATCH_SIZE)
dataset_iterator = parsed_train_dataset.make_one_shot_iterator()

variable_dataset = dataset_iterator.get_next()

## Model Design

In [22]:
enc_len = 128
age_len = 10
img_shape = (32, 32, 3)
width, height, depth = (32, 32, 3)
img_len = np.prod(img_shape)
latent_dim = enc_len + age_len + img_len
noise_len = 100
input_dim = enc_len + age_len + noise_len
cond_len = enc_len + age_len


def build_discriminator():
    conv = keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=16,
            kernel_size=(3, 3),
            strides=2,
            input_shape=img_shape
        ),
        keras.layers.Activation(tf.nn.leaky_relu),

        # conv block 2
        keras.layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=3,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
    ])
    
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(128, input_shape=(latent_dim,)),
        keras.layers.Activation(tf.nn.relu),
        
        # output
        keras.layers.Dense(1),
        keras.layers.Activation(tf.nn.sigmoid),
    ])
    
    model.summary()
    
    # condition
    c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # image
    z = keras.layers.Input(shape=img_shape)
    
    # convolution
    zout = conv(z)
    
    # flatten image
    z_flat = keras.layers.Flatten()(zout)
    
    # concatenation
    inputs = keras.layers.concatenate([c1, c2, z_flat])
    
    # real or fake
    outputs = model(inputs)
    
    return keras.models.Model([c1, c2, z], outputs)


def build_generator():
    
    conv = keras.Sequential([
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=16,
            kernel_size=(3, 3),
            strides=1,
            input_shape=(11, 11, 1)
        ),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(2, 2),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),

        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=3,
            kernel_size=(5, 5),
            strides=1
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])
    
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(121, input_shape=(input_dim,)),
        keras.layers.Activation(tf.nn.relu),
        
        # reshape 1d to 3d
        keras.layers.Reshape((11, 11, 1))
    ])
    
    model.summary()
    
    # condition
    c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # noise
    x = keras.layers.Input(shape=(noise_len,))

    # concatenation
    inputs = keras.layers.concatenate([c1, c2, x])
    
    # flat dense output
    out_1 = model(inputs)
    
    # transpose conv output
    outputs = conv(out_1)
    
    return keras.models.Model([c1, c2, x], outputs)


discriminator = build_discriminator()
generator = build_generator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 15, 15, 16)        448       
_________________________________________________________________
activation_129 (Activation)  (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 7, 7, 32)          4640      
_________________________________________________________________
activation_130 (Activation)  (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 3, 3, 3)           867       
_________________________________________________________________
activation_131 (Activation)  (None, 3, 3, 3)           0         
Total params: 5,955
Trainable params: 5,955
Non-trainable params: 0
_________________________________________________________________
________

In [23]:
generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_85 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
input_86 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
input_87 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
concatenate_29 (Concatenate)    (None, 238)          0           input_85[0][0]                   
                                                                 input_86[0][0]                   
          

In [24]:
discriminator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_84 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
sequential_55 (Sequential)      (None, 3, 3, 3)      5955        input_84[0][0]                   
__________________________________________________________________________________________________
input_82 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
input_83 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
flatten_15

## Generative Adversarial Network

In [26]:
GLR = 0.0002  # generator
DLR = 0.0002  # discriminator


discriminator.compile(
    optimizer=keras.optimizers.Adam(DLR, 0.5),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy']
)


# condition
c1 = keras.layers.Input(shape=(enc_len,))
c2 = keras.layers.Input(shape=(age_len,))

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator([c1, c2, x])
out = discriminator([c1, c2, z])

# GAN
gan = keras.models.Model(inputs=[c1, c2, x], outputs=out)

gan.compile(
    optimizer=keras.optimizers.Adam(GLR , 0.5),
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy'])

In [27]:
gan.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_90 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
input_91 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
input_92 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
model_29 (Model)                (None, 32, 32, 3)    40250       input_90[0][0]                   
                                                                 input_91[0][0]                   
          

## Visualization Method

In [28]:
# from google.colab import drive
import os


# drive.mount('/content/gdrive', force_remount=True)

root_path = './'
tgt_pth = os.path.join(root_path, 'visualize_age-cgan-v1')

if not os.path.exists(tgt_pth):
  os.mkdir(tgt_pth)

In [29]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(8, 8, figsize=(40, 36))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(z_real[r_real * 8 + col])
            else:
                cell.imshow(z_fake[r_fake * 8 + col])

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    
    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

## Load Batch

In [12]:
def load_noise():
    
    y_true = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_true = y_true.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            # concatenate face + age + noise
            c1 = row['enc']
            c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            
            yield c1, c2, x, y_true


def load_batch():
    
    y_fake = tf.zeros((BATCH_SIZE,))
    y_true = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_fake = y_fake.eval()
        y_true = y_true.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            c1 = row['enc']
            c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            z_fake = generator.predict([c1, c2, x])

            # real data
            c1 = row['enc']
            c2 = tf.cast(row['age'], tf.float32).eval()
            z_real = tf.reshape(tf.io.decode_raw(row['img'], tf.uint8), (-1, width, height, depth)).eval()
                        
            yield c1, c2, x, z_fake, y_fake, z_real, y_true

## Train Model

In [ ]:
BATCH_SIZE = 32
EPOCHS = 4000
STEPS = 1  # 60000 // BATCH_SIZE


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


disc_itr = load_batch()
gen_itr = load_noise()


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS * 4):
        
        c1, c2, x, z_fake, y_fake, z_real, y_real = next(disc_itr)
    
        # train
        loss_2, acc_2 = discriminator.train_on_batch([c1, c2, z_real], y_real)
        loss_1, acc_1 = discriminator.train_on_batch([c1, c2, z_fake], y_fake)

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

        c1, c2, x, y_true = next(gen_itr)

        # train
        loss_1, acc_1 = gan.train_on_batch([c1, c2, x], y_true)

        loss.append(loss_1)
        acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("Epoch: {}, Steps: {}, Discriminator Accuracy: %{:.2f}, GAN Accuracy: %{:.2f}".format(
          e,
          STEPS,
          train_acc_d[-1] * 100,
          train_acc_g[-1] * 100
      ))

    if e % 50 == 0:
        ## visualize results
        c1, c2, x, z_fake, y_fake, z_real, y_real = next(disc_itr)
        visualizeGAN(e, z_real, z_fake)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch: 0, Steps: 1, Discriminator Accuracy: %50.39, GAN Accuracy: %28.12
Epoch: 1, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %15.62
Epoch: 2, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %9.38
Epoch: 3, Steps: 1, Discriminator Accuracy: %51.17, GAN Accuracy: %28.12
Epoch: 4, Steps: 1, Discriminator Accuracy: %53.91, GAN Accuracy: %12.50
Epoch: 5, Steps: 1, Discriminator Accuracy: %54.30, GAN Accuracy: %21.88
Epoch: 6, Steps: 1, Discriminator Accuracy: %56.25, GAN Accuracy: %37.50
Epoch: 7, Steps: 1, Discriminator Accuracy: %59.77, GAN Accuracy: %12.50
Epoch: 8, Steps: 1, Discriminator Accuracy: %61.33, GAN Accuracy: %15.62
Epoch: 9, Steps: 1, Discriminator Accuracy: %61.33, GAN Accuracy: %12.50
Epoch: 10, Steps: 1, Discriminator Accuracy: %67.19, GAN Accuracy: %21.88
Epoch: 11, Steps: 1, Discriminator Accuracy: %73.83, GAN Accuracy: %21.88
Epoch: 12, Steps: 1, Discriminator Accuracy: %71.09, GAN Accuracy: %21.88
Epoch: 13, Steps: 1, Discriminator Accuracy: %79.

Epoch: 112, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 113, Steps: 1, Discriminator Accuracy: %99.61, GAN Accuracy: %0.00
Epoch: 114, Steps: 1, Discriminator Accuracy: %99.61, GAN Accuracy: %0.00
Epoch: 115, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 116, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 117, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 118, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 119, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 120, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 121, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 122, Steps: 1, Discriminator Accuracy: %99.61, GAN Accuracy: %0.00
Epoch: 123, Steps: 1, Discriminator Accuracy: %99.61, GAN Accuracy: %0.00
Epoch: 124, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 125, Steps: 1, Discrim

Epoch: 223, Steps: 1, Discriminator Accuracy: %99.61, GAN Accuracy: %0.00
Epoch: 224, Steps: 1, Discriminator Accuracy: %99.61, GAN Accuracy: %0.00
Epoch: 225, Steps: 1, Discriminator Accuracy: %99.22, GAN Accuracy: %0.00
Epoch: 226, Steps: 1, Discriminator Accuracy: %99.61, GAN Accuracy: %3.12
Epoch: 227, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 228, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 229, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 230, Steps: 1, Discriminator Accuracy: %99.61, GAN Accuracy: %0.00
Epoch: 231, Steps: 1, Discriminator Accuracy: %98.83, GAN Accuracy: %0.00
Epoch: 232, Steps: 1, Discriminator Accuracy: %100.00, GAN Accuracy: %0.00
Epoch: 233, Steps: 1, Discriminator Accuracy: %99.22, GAN Accuracy: %0.00
Epoch: 234, Steps: 1, Discriminator Accuracy: %99.61, GAN Accuracy: %0.00
Epoch: 235, Steps: 1, Discriminator Accuracy: %99.61, GAN Accuracy: %0.00
Epoch: 236, Steps: 1, Discriminato

...

Epoch: 1219, Steps: 1, Discriminator Accuracy: %90.23, GAN Accuracy: %12.50
Epoch: 1220, Steps: 1, Discriminator Accuracy: %86.33, GAN Accuracy: %3.12
Epoch: 1221, Steps: 1, Discriminator Accuracy: %84.77, GAN Accuracy: %12.50
Epoch: 1222, Steps: 1, Discriminator Accuracy: %86.33, GAN Accuracy: %18.75
Epoch: 1223, Steps: 1, Discriminator Accuracy: %89.84, GAN Accuracy: %6.25
Epoch: 1224, Steps: 1, Discriminator Accuracy: %89.84, GAN Accuracy: %3.12
Epoch: 1225, Steps: 1, Discriminator Accuracy: %89.06, GAN Accuracy: %3.12
Epoch: 1226, Steps: 1, Discriminator Accuracy: %86.33, GAN Accuracy: %6.25
Epoch: 1227, Steps: 1, Discriminator Accuracy: %86.72, GAN Accuracy: %9.38
Epoch: 1228, Steps: 1, Discriminator Accuracy: %89.06, GAN Accuracy: %0.00
Epoch: 1229, Steps: 1, Discriminator Accuracy: %84.38, GAN Accuracy: %6.25
Epoch: 1230, Steps: 1, Discriminator Accuracy: %84.77, GAN Accuracy: %9.38
Epoch: 1231, Steps: 1, Discriminator Accuracy: %89.06, GAN Accuracy: %9.38
Epoch: 1232, Steps: 1,

Epoch: 1328, Steps: 1, Discriminator Accuracy: %84.38, GAN Accuracy: %9.38
Epoch: 1329, Steps: 1, Discriminator Accuracy: %89.06, GAN Accuracy: %3.12
Epoch: 1330, Steps: 1, Discriminator Accuracy: %87.50, GAN Accuracy: %0.00
Epoch: 1331, Steps: 1, Discriminator Accuracy: %90.62, GAN Accuracy: %6.25
Epoch: 1332, Steps: 1, Discriminator Accuracy: %82.03, GAN Accuracy: %9.38
Epoch: 1333, Steps: 1, Discriminator Accuracy: %87.50, GAN Accuracy: %0.00
Epoch: 1334, Steps: 1, Discriminator Accuracy: %89.06, GAN Accuracy: %3.12
Epoch: 1335, Steps: 1, Discriminator Accuracy: %92.97, GAN Accuracy: %9.38
Epoch: 1336, Steps: 1, Discriminator Accuracy: %90.23, GAN Accuracy: %6.25
Epoch: 1337, Steps: 1, Discriminator Accuracy: %94.14, GAN Accuracy: %3.12
Epoch: 1338, Steps: 1, Discriminator Accuracy: %81.64, GAN Accuracy: %9.38
Epoch: 1339, Steps: 1, Discriminator Accuracy: %83.20, GAN Accuracy: %3.12
Epoch: 1340, Steps: 1, Discriminator Accuracy: %86.33, GAN Accuracy: %18.75
Epoch: 1341, Steps: 1, D

Epoch: 1437, Steps: 1, Discriminator Accuracy: %87.50, GAN Accuracy: %9.38
Epoch: 1438, Steps: 1, Discriminator Accuracy: %91.02, GAN Accuracy: %12.50
Epoch: 1439, Steps: 1, Discriminator Accuracy: %89.45, GAN Accuracy: %12.50
Epoch: 1440, Steps: 1, Discriminator Accuracy: %88.67, GAN Accuracy: %0.00
Epoch: 1441, Steps: 1, Discriminator Accuracy: %73.44, GAN Accuracy: %0.00
Epoch: 1442, Steps: 1, Discriminator Accuracy: %86.72, GAN Accuracy: %0.00
Epoch: 1443, Steps: 1, Discriminator Accuracy: %87.89, GAN Accuracy: %6.25
Epoch: 1444, Steps: 1, Discriminator Accuracy: %83.59, GAN Accuracy: %12.50
Epoch: 1445, Steps: 1, Discriminator Accuracy: %91.80, GAN Accuracy: %3.12
Epoch: 1446, Steps: 1, Discriminator Accuracy: %87.11, GAN Accuracy: %6.25
Epoch: 1447, Steps: 1, Discriminator Accuracy: %85.55, GAN Accuracy: %9.38
Epoch: 1448, Steps: 1, Discriminator Accuracy: %90.62, GAN Accuracy: %3.12
Epoch: 1449, Steps: 1, Discriminator Accuracy: %85.94, GAN Accuracy: %6.25
Epoch: 1450, Steps: 1,

Epoch: 1546, Steps: 1, Discriminator Accuracy: %83.98, GAN Accuracy: %12.50
Epoch: 1547, Steps: 1, Discriminator Accuracy: %87.11, GAN Accuracy: %6.25
Epoch: 1548, Steps: 1, Discriminator Accuracy: %85.16, GAN Accuracy: %6.25
Epoch: 1549, Steps: 1, Discriminator Accuracy: %76.17, GAN Accuracy: %12.50
Epoch: 1550, Steps: 1, Discriminator Accuracy: %82.81, GAN Accuracy: %3.12
Epoch: 1551, Steps: 1, Discriminator Accuracy: %88.28, GAN Accuracy: %3.12
Epoch: 1552, Steps: 1, Discriminator Accuracy: %92.58, GAN Accuracy: %6.25
Epoch: 1553, Steps: 1, Discriminator Accuracy: %90.62, GAN Accuracy: %0.00
Epoch: 1554, Steps: 1, Discriminator Accuracy: %85.16, GAN Accuracy: %0.00
Epoch: 1555, Steps: 1, Discriminator Accuracy: %85.94, GAN Accuracy: %21.88
Epoch: 1556, Steps: 1, Discriminator Accuracy: %89.84, GAN Accuracy: %12.50
Epoch: 1557, Steps: 1, Discriminator Accuracy: %89.06, GAN Accuracy: %3.12
Epoch: 1558, Steps: 1, Discriminator Accuracy: %92.97, GAN Accuracy: %9.38
Epoch: 1559, Steps: 1

Epoch: 1655, Steps: 1, Discriminator Accuracy: %89.84, GAN Accuracy: %3.12
Epoch: 1656, Steps: 1, Discriminator Accuracy: %88.67, GAN Accuracy: %3.12
Epoch: 1657, Steps: 1, Discriminator Accuracy: %90.23, GAN Accuracy: %9.38
Epoch: 1658, Steps: 1, Discriminator Accuracy: %87.11, GAN Accuracy: %6.25
Epoch: 1659, Steps: 1, Discriminator Accuracy: %87.11, GAN Accuracy: %6.25
Epoch: 1660, Steps: 1, Discriminator Accuracy: %91.80, GAN Accuracy: %6.25
Epoch: 1661, Steps: 1, Discriminator Accuracy: %89.84, GAN Accuracy: %6.25
Epoch: 1662, Steps: 1, Discriminator Accuracy: %89.06, GAN Accuracy: %6.25
Epoch: 1663, Steps: 1, Discriminator Accuracy: %89.06, GAN Accuracy: %9.38
Epoch: 1664, Steps: 1, Discriminator Accuracy: %85.94, GAN Accuracy: %0.00
Epoch: 1665, Steps: 1, Discriminator Accuracy: %85.55, GAN Accuracy: %6.25
Epoch: 1666, Steps: 1, Discriminator Accuracy: %87.50, GAN Accuracy: %3.12
Epoch: 1667, Steps: 1, Discriminator Accuracy: %92.97, GAN Accuracy: %0.00
Epoch: 1668, Steps: 1, Di

## Plot Loss

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

## Plot Accuracy

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();

In [ ]:
generator.save('./mnist-gen.h5')

In [ ]:
discriminator.save('./mnist-disc.h5')

In [ ]:
gan.save('./mnist-gan.h5')